# XGBoost Play Prediction Model Training



In [ ]:
import datetime

import cupy as cp
import xgboost as xgb
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder

import nflpp

Use cuda for accelerated training if available.

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

## Process Data

### Load the play-by-play data
The data for this project is provided by the [nfl_data_py](https://github.com/nflverse/nfl_data_py) python library. The dataset covers the NFL seasons from 1999 to 2023, so we'll load all years in that range. A function from this project's `nflpp` module is used to ensure the data is cached and loaded from cache at all times for better peformance.

In [ ]:
%%capture
pbp_data = nflpp.load_pbp_data(range(1999, 2024))

### Filter excess data
The vast majority of plays called in an NFL game are either runs or passes. Other types of plays are either forced by game rules (kickoffs) or almost always shown pre-snap by formation (field goals and punts). For these reasons, the play prediction model will focus only on play calls involving a run or pass.

In [ ]:
processed_data = pbp_data[pbp_data["play_type"].isin(["run", "pass"])]

print(f"Run/pass filtered shape: {processed_data.shape}")

The play-by-play data has a large amount of columns that can't be used for this project. The goal is to have the model be used to predict the next play call within the 40 seconds the team has to snap the ball. Many columns, such as `yards_gained`, contain information about the outcome of the play. Others contain data that would not be able to be processed pre-snap, like `wp` (win percentage). Because of this, the data will be filtered down to the most important columns that can be easily gathered pre-snap.

In [ ]:
columns = [
    # Target
    "play_type",
    # Features
    "drive",
    "qtr",
    "quarter_seconds_remaining",
    "down",
    "ydstogo",
    "yardline_100",
    "shotgun",
    "no_huddle",
    "score_differential",
    "spread_line",
    "season",
    "posteam",
    # Used to create poscoach
    "home_coach",
    "away_coach",
    "home_team",
    "away_team",
]
processed_data = processed_data[columns]
print(f"Run/pass filtered shape: {processed_data.shape}")

### Create new column

While the data contains a variable for the team currently in possession of the ball, no such variable exists for the coach. This will create a new column for the possessing team's coach, and then remove the columns used its creation.

In [ ]:
processed_data.loc[:, "poscoach"] = processed_data.apply(
    lambda row: row["home_coach"]
    if row["posteam"] == row["home_team"]
    else row["away_coach"],
    axis=1,
)
processed_data = processed_data.drop(
    columns=["home_coach", "away_coach", "home_team", "away_team"]
)
print(f"Post-poscoach shape: {processed_data.shape}")

### Encode descriptive features

XGBoost needs data to be numerical, so the variables that are present in the dataset need to be encoded to integer values.

In [ ]:
for column in ("posteam", "poscoach", "play_type"):
    processed_data[column] = LabelEncoder().fit_transform(processed_data[column])

### Partition data for training

In order to train the model, the data must be split for evaluation after training. Since the use-case of the model will be to predict future results based on previous data, the final season will be used as the evaluation dataset.

In [ ]:
train_data = processed_data[processed_data["season"] < 2022]
test_data = processed_data[processed_data["season"] == 2022]
validation_data = processed_data[processed_data["season"] == 2023]

To be used for training functions, the training and testing datasets must be further split into the features and the target.

In [ ]:
train_features = train_data.drop("play_type", axis=1)
train_target = train_data["play_type"]
test_features = train_data.drop("play_type", axis=1)
test_target = train_data["play_type"]

In [ ]:
if device == "cuda":
    train_features = cp.asarray(train_features)
    train_target = cp.asarray(train_target)
    test_features = cp.asarray(test_features)
    test_target = cp.asarray(test_target)

In [ ]:
dtrain = xgb.DMatrix(train_features, label=train_target)
dtest = xgb.DMatrix(test_features, label=test_target)

eval_set = [(dtrain, "train"), (dtest, "test")]

## Model Training



In [ ]:
model_parameters = {
    "objective": "binary:logistic",
    "device": device,
    "eval_metric": ["auc", "error", "logloss"],
}
model = xgb.train(model_parameters, dtrain, num_boost_round=100)

In [ ]:
test_preds = (model.predict(dtest) > 0.5).astype(int)

accuracy = accuracy_score(test_target, test_preds)
f1 = f1_score(test_target, test_preds)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")

### Saving the model

After training, the XGBoost model is saved in JSON file found within the /models directory.

In [ ]:
models_directory = "../models"

version = "1.1.1"
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"{models_directory}/xgboost_v{version}_{timestamp}_{accuracy:.4f}.json"

model.save_model(filename)

## Model Analysis

The rest of the notebook is devoted to evaluating the performance of the trained model in greater detail. These model analyses are used to compare each iteration of the model in order to focus future developments.

In [ ]:
nflpp.plot_confusion_matrix(test_target, test_preds)

In [ ]:
probabilities = clf.predict_proba(test_features)
nflpp.plot_roc_curve(test_target, probabilities)

In [ ]:
features = train_data.drop("play_type", axis=1).columns.tolist()
clf.get_booster().feature_names = features
plot_importance(clf)
plt.show()

In [ ]:
results = clf.evals_result()
nflpp.plot_learning_curves(results)